## Installation

In [ ]:
pip install sklearn

## Import libraries' 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn as sk
import seaborn as sns

In [ ]:
import psycopg2
import psycopg2.extras
import pandas.io.sql as sqlio

#import warnings
#warnings.filterwarnings('ignore')

## Load Dataset From CSV

In [6]:
cus = pd.read_csv("dataset/Coustomer_dim.csv",low_memory=False)
fact = pd.read_csv("dataset/Fact_data.csv",low_memory=False)
item = pd.read_csv("dataset/Item_dim.csv",low_memory=False)
store = pd.read_csv("dataset/Store_dim.csv",low_memory=False)
time = pd.read_csv("dataset/Time_dim.csv",low_memory=False)
trans = pd.read_csv("dataset/Trans_dim.csv",low_memory=False)

## Load Dataset From Postgres

**Connect to the database**

In [32]:
conn = psycopg2.connect(database="ecomdb",
                                           user = "postgres",
                                           password = "afra1234",
                                           host = "127.0.0.1",
                                           port = "5432")
print("successfully connected to database")  

successfully connected to database


In [97]:
cur = conn.cursor()
select_stmt = "select i.item_key,s.store_key, t.payment_key,t.trans_type, i.man_country, i.descript, i.supplier, i.unit, td.year,td.month,td.day,td.hour,td.week, s.district, f.total_price " \
"from ecom_schema.fact_table f " \
"join ecom_schema.item_dim i on i.item_key=f.item_key " \
"join ecom_schema.time_dim td on td.time_key = f.time_key " \
"join ecom_schema.store_dim s on s.store_key = f.store_key " \
"join ecom_schema.trans_dim t on t.payment_key=f.payment_key " \

cur.execute(select_stmt)
records = cur.fetchall()

In [98]:
df = pd.DataFrame(list(records), columns=['item_key', 'store_key','payment_key','trans_type',
                                          'man_country','descript','supplier','unit',
                                          'year','month','day','hour','week','district','total_price'])
df.head(20)

,item_key,store_key,payment_key,trans_type,man_country,descript,supplier,unit,year,month,day,hour,week,district,total_price
0,I00131,S0086,P032,card,Bangladesh,Dishware - Bowls,DENIMACH LTD,ct,2020,2,1,6,1st Week,RANGPUR,112.0
1,I00017,S00164,P025,card,Cambodia,Beverage - Soda,NINGBO SEDUNO IMP & EXP CO.LTD,cans,2016,10,16,5,3rd Week,SIRAJGANJ,74.25
2,I00133,S00640,P019,card,United States,Dishware - Cups Cold,MAESA SAS,ct,2018,2,13,1,2nd Week,DHAKA,150.0
3,I00023,S0032,P030,card,Cambodia,Beverage - Soda,NINGBO SEDUNO IMP & EXP CO.LTD,cans,2017,11,29,16,4th Week,SYLHET,67.5
4,I00091,S00455,P015,card,Cambodia,Beverage Water,NINGBO SEDUNO IMP & EXP CO.LTD,bottles,2018,4,9,16,2nd Week,COMILLA,15.0
5,I00155,S00616,P013,card,India,Food - Chips,Indo Count Industries Ltd,bags,2016,11,26,10,4th Week,DHAKA,153.0
6,I00066,S00142,P028,card,Cambodia,Beverage - Gatorade,NINGBO SEDUNO IMP & EXP CO.LTD,bottles,2017,9,8,9,2nd Week,PABNA,20.0
7,I00218,S00395,P016,card,Netherlands,Food - Nuts,Bolsius Boxmeer,ct,2015,7,6,9,1st Week,TANGAIL,18.0
8,I00246,S0037,P029,card,Germany,Kitchen Supplies,Friedola 1888 GmbH,ct,2017,10,27,12,4th Week,SYLHET,60.0
9,I00204,S00643,P033,card,Finland,Food - Healthy,HARDFORD AB,ct,2014,7,23,22,4th Week,DHAKA,85.0


In [99]:
df.shape

(1000000, 15)

In [100]:
df.to_csv("ecom_sales.csv",index=False)

#### Load DataSet

In [101]:
df= pd.read_csv("ecom_sales.csv")
df

,item_key,store_key,payment_key,trans_type,man_country,descript,supplier,unit,year,month,day,hour,week,district,total_price
0,I00131,S0086,P032,card,Bangladesh,Dishware - Bowls,DENIMACH LTD,ct,2020,2,1,6,1st Week,RANGPUR,112.00
1,I00017,S00164,P025,card,Cambodia,Beverage - Soda,NINGBO SEDUNO IMP & EXP CO.LTD,cans,2016,10,16,5,3rd Week,SIRAJGANJ,74.25
2,I00133,S00640,P019,card,United States,Dishware - Cups Cold,MAESA SAS,ct,2018,2,13,1,2nd Week,DHAKA,150.00
3,I00023,S0032,P030,card,Cambodia,Beverage - Soda,NINGBO SEDUNO IMP & EXP CO.LTD,cans,2017,11,29,16,4th Week,SYLHET,67.50
4,I00091,S00455,P015,card,Cambodia,Beverage Water,NINGBO SEDUNO IMP & EXP CO.LTD,bottles,2018,4,9,16,2nd Week,COMILLA,15.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,I00052,S00654,P002,card,poland,Beverage - Energy/Protein,CHROMADURLIN S.A.S,cans,2014,4,21,8,3rd Week,DHAKA,120.00
999996,I00011,S00587,P029,card,Germany,Beverage - Soda,Friedola 1888 GmbH,cans,2017,6,5,19,1st Week,DHAKA,27.00
999997,I00191,S00294,P020,card,India,Food - Healthy,Indo Count Industries Ltd,bars,2017,8,2,3,1st Week,FARIDPUR,216.00
999998,I00109,S0001,P014,card,Germany,Coffee Ground,Friedola 1888 GmbH,oz,2015,1,13,14,2nd Week,HABIGANJ,51.00


In [102]:
# Make a copy of the original DataFrame
df_tmp = df.copy()

In [103]:
# Check for missing categories and different datatypes
df_tmp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 15 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   item_key     1000000 non-null  object 
 1   store_key    1000000 non-null  object 
 2   payment_key  1000000 non-null  object 
 3   trans_type   1000000 non-null  object 
 4   man_country  1000000 non-null  object 
 5   descript     1000000 non-null  object 
 6   supplier     1000000 non-null  object 
 7   unit         1000000 non-null  object 
 8   year         1000000 non-null  int64  
 9   month        1000000 non-null  int64  
 10  day          1000000 non-null  int64  
 11  hour         1000000 non-null  int64  
 12  week         1000000 non-null  object 
 13  district     1000000 non-null  object 
 14  total_price  1000000 non-null  float64
dtypes: float64(1), int64(4), object(10)
memory usage: 114.4+ MB


## Convert strings to categories
One way to help turn all of our data into numbers is to convert the columns with the string datatype into a category datatype.

To do this we can use the pandas types API which allows us to interact and manipulate the types of data.

In [104]:
df_tmp.head().T

,0,1,2,3,4
item_key,I00131,I00017,I00133,I00023,I00091
store_key,S0086,S00164,S00640,S0032,S00455
payment_key,P032,P025,P019,P030,P015
trans_type,card,card,card,card,card
man_country,Bangladesh,Cambodia,United States,Cambodia,Cambodia
descript,Dishware - Bowls,Beverage - Soda,Dishware - Cups Cold,Beverage - Soda,Beverage Water
supplier,DENIMACH LTD,NINGBO SEDUNO IMP & EXP CO.LTD,MAESA SAS,NINGBO SEDUNO IMP & EXP CO.LTD,NINGBO SEDUNO IMP & EXP CO.LTD
unit,ct,cans,ct,cans,bottles
year,2020,2016,2018,2017,2018
month,2,10,2,11,4


In [105]:
pd.api.types.is_string_dtype(df_tmp["descript"])

True

In [106]:
# These columns contain strings
for label, content in df_tmp.items():
    if pd.api.types.is_string_dtype(content):
        print(label)

item_key
store_key
payment_key
trans_type
man_country
descript
supplier
unit
week
district


In [93]:
random_dict = {"key1": "hello",
               "key2": "world!"}

for key, value in random_dict.items():
    print(f"This is a key: {key}")
    print(f"This is a value: {value}")

This is a key: key1
This is a value: hello
This is a key: key2
This is a value: world!


In [108]:
# Turn all of the string values into category values
for label, content in df_tmp.items():
    if pd.api.types.is_string_dtype(content):
        df_tmp[label] = content.astype("category").cat.as_ordered()

In [109]:
df_tmp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 15 columns):
 #   Column       Non-Null Count    Dtype   
---  ------       --------------    -----   
 0   item_key     1000000 non-null  category
 1   store_key    1000000 non-null  category
 2   payment_key  1000000 non-null  category
 3   trans_type   1000000 non-null  category
 4   man_country  1000000 non-null  category
 5   descript     1000000 non-null  category
 6   supplier     1000000 non-null  category
 7   unit         1000000 non-null  category
 8   year         1000000 non-null  int64   
 9   month        1000000 non-null  int64   
 10  day          1000000 non-null  int64   
 11  hour         1000000 non-null  int64   
 12  week         1000000 non-null  category
 13  district     1000000 non-null  category
 14  total_price  1000000 non-null  float64 
dtypes: category(10), float64(1), int64(4)
memory usage: 49.6 MB


In [110]:
df_tmp.district.cat.categories

Index(['BAGERHAT', 'BANDARBAN', 'BARGUNA', 'BARISAL', 'BHOLA', 'BOGRA',
       'BRAHMANBARIA', 'CHANDPUR', 'CHAPAI NABABGANJ', 'CHITTAGONG',
       'CHUADANGA', 'COMILLA', 'COX'S BAZAR', 'DHAKA', 'DINAJPUR', 'FARIDPUR',
       'FENI', 'GAIBANDHA', 'GAZIPUR', 'GOPALGANJ', 'HABIGANJ', 'JAMALPUR',
       'JESSORE', 'JHALOKATI', 'JHENAIDAH', 'JOYPURHAT', 'KHAGRACHHARI',
       'KHULNA', 'KISHOREGONJ', 'KURIGRAM', 'KUSHTIA', 'LAKSHMIPUR',
       'LALMONIRHAT', 'MADARIPUR', 'MAGURA', 'MANIKGANJ', 'MAULVIBAZAR',
       'MEHERPUR', 'MUNSHIGANJ', 'MYMENSINGH', 'NAOGAON', 'NARAIL',
       'NARAYANGANJ', 'NARSINGDI', 'NATORE', 'NETRAKONA', 'NILPHAMARI',
       'NOAKHALI', 'PABNA', 'PANCHAGARH', 'PATUAKHALI', 'PIROJPUR', 'RAJBARI',
       'RAJSHAHI', 'RANGAMATI', 'RANGPUR', 'SATKHIRA', 'SHARIATPUR', 'SHERPUR',
       'SIRAJGANJ', 'SUNAMGANJ', 'SYLHET', 'TANGAIL', 'THAKURGAON'],
      dtype='object')

In [111]:
df_tmp.district.cat.codes

0         55
1         59
2         13
3         61
4         11
          ..
999995    13
999996    13
999997    15
999998    20
999999    51
Length: 1000000, dtype: int8

**All of our data is categorical and thus we can now turn the categories into numbers**

In [112]:
# Save preprocessed data
df_tmp.to_csv("dataset/ecom_processed.csv",index=False)

## Import preprocessed data

In [2]:
df_tmp = pd.read_csv("dataset/ecom_processed.csv",low_memory=False)
df_tmp.head().T

,0,1,2,3,4
item_key,131,17,133,23,91
store_key,86,164,640,32,455
payment_key,32,25,19,30,15
trans_type,card,card,card,card,card
man_country,Bangladesh,Cambodia,United States,Cambodia,Cambodia
descript,Dishware - Bowls,Beverage - Soda,Dishware - Cups Cold,Beverage - Soda,Beverage Water
supplier,DENIMACH LTD,NINGBO SEDUNO IMP & EXP CO.LTD,MAESA SAS,NINGBO SEDUNO IMP & EXP CO.LTD,NINGBO SEDUNO IMP & EXP CO.LTD
unit,ct,cans,ct,cans,bottles
year,2020,2016,2018,2017,2018
month,2,10,2,11,4


In [13]:
#df_tmp["item_key"] =''.join(c for c in df_tmp["item_key"] if not c.isdigit())

## Convert Key Into STring

In [ ]:
df_tmp['item_key'] = df_tmp['item_key'].map(lambda x: x.lstrip("I"))
df_tmp['item_key'].head()

In [ ]:
df_tmp['store_key'] = df_tmp['store_key'].map(lambda x: x.lstrip("S"))
df_tmp['store_key'].head()

In [23]:
df_tmp['payment_key'] = df_tmp['payment_key'].map(lambda x: x.lstrip("P"))
df_tmp['payment_key'].head()

0    032
1    025
2    019
3    030
4    015
Name: payment_key, dtype: object

In [5]:
df_tmp.head()

,item_key,store_key,payment_key,trans_type,man_country,descript,supplier,unit,year,month,day,hour,week,district,total_price
0,131,86,32,card,Bangladesh,Dishware - Bowls,DENIMACH LTD,ct,2020,2,1,6,1st Week,RANGPUR,112.00
1,17,164,25,card,Cambodia,Beverage - Soda,NINGBO SEDUNO IMP & EXP CO.LTD,cans,2016,10,16,5,3rd Week,SIRAJGANJ,74.25
2,133,640,19,card,United States,Dishware - Cups Cold,MAESA SAS,ct,2018,2,13,1,2nd Week,DHAKA,150.00
3,23,32,30,card,Cambodia,Beverage - Soda,NINGBO SEDUNO IMP & EXP CO.LTD,cans,2017,11,29,16,4th Week,SYLHET,67.50
4,91,455,15,card,Cambodia,Beverage Water,NINGBO SEDUNO IMP & EXP CO.LTD,bottles,2018,4,9,16,2nd Week,COMILLA,15.00


In [25]:
# Save preprocessed data
df_tmp.to_csv("dataset/ecom_processed.csv",index=False)

In [4]:
df_tmp = pd.read_csv("dataset/ecom_processed.csv",low_memory=False)
df_tmp.head().T

,0,1,2,3,4
item_key,131,17,133,23,91
store_key,86,164,640,32,455
payment_key,32,25,19,30,15
trans_type,card,card,card,card,card
man_country,Bangladesh,Cambodia,United States,Cambodia,Cambodia
descript,Dishware - Bowls,Beverage - Soda,Dishware - Cups Cold,Beverage - Soda,Beverage Water
supplier,DENIMACH LTD,NINGBO SEDUNO IMP & EXP CO.LTD,MAESA SAS,NINGBO SEDUNO IMP & EXP CO.LTD,NINGBO SEDUNO IMP & EXP CO.LTD
unit,ct,cans,ct,cans,bottles
year,2020,2016,2018,2017,2018
month,2,10,2,11,4


In [6]:
df_tmp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 15 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   item_key     1000000 non-null  int64  
 1   store_key    1000000 non-null  int64  
 2   payment_key  1000000 non-null  int64  
 3   trans_type   1000000 non-null  object 
 4   man_country  1000000 non-null  object 
 5   descript     1000000 non-null  object 
 6   supplier     1000000 non-null  object 
 7   unit         1000000 non-null  object 
 8   year         1000000 non-null  int64  
 9   month        1000000 non-null  int64  
 10  day          1000000 non-null  int64  
 11  hour         1000000 non-null  int64  
 12  week         1000000 non-null  object 
 13  district     1000000 non-null  object 
 14  total_price  1000000 non-null  float64
dtypes: float64(1), int64(7), object(7)
memory usage: 114.4+ MB


In [7]:
# These columns contain strings
for label, content in df_tmp.items():
    if pd.api.types.is_string_dtype(content):
        print(label)

trans_type
man_country
descript
supplier
unit
week
district


In [8]:
# If you're wondering what df.items() does, let's use a dictionary as an example
random_dict = {"key1": "hello",
               "key2": "world!"}

for key, value in random_dict.items():
    print(f"This is a key: {key}")
    print(f"This is a value: {value}")

This is a key: key1
This is a value: hello
This is a key: key2
This is a value: world!


In [9]:
# This will turn all of the string values into category values
for label, content in df_tmp.items():
    if pd.api.types.is_string_dtype(content):
        df_tmp[label] = content.astype("category").cat.as_ordered()

In [10]:
df_tmp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 15 columns):
 #   Column       Non-Null Count    Dtype   
---  ------       --------------    -----   
 0   item_key     1000000 non-null  int64   
 1   store_key    1000000 non-null  int64   
 2   payment_key  1000000 non-null  int64   
 3   trans_type   1000000 non-null  category
 4   man_country  1000000 non-null  category
 5   descript     1000000 non-null  category
 6   supplier     1000000 non-null  category
 7   unit         1000000 non-null  category
 8   year         1000000 non-null  int64   
 9   month        1000000 non-null  int64   
 10  day          1000000 non-null  int64   
 11  hour         1000000 non-null  int64   
 12  week         1000000 non-null  category
 13  district     1000000 non-null  category
 14  total_price  1000000 non-null  float64 
dtypes: category(7), float64(1), int64(7)
memory usage: 67.7 MB


In [11]:
df_tmp.district.cat.categories

Index(['BAGERHAT', 'BANDARBAN', 'BARGUNA', 'BARISAL', 'BHOLA', 'BOGRA',
       'BRAHMANBARIA', 'CHANDPUR', 'CHAPAI NABABGANJ', 'CHITTAGONG',
       'CHUADANGA', 'COMILLA', 'COX'S BAZAR', 'DHAKA', 'DINAJPUR', 'FARIDPUR',
       'FENI', 'GAIBANDHA', 'GAZIPUR', 'GOPALGANJ', 'HABIGANJ', 'JAMALPUR',
       'JESSORE', 'JHALOKATI', 'JHENAIDAH', 'JOYPURHAT', 'KHAGRACHHARI',
       'KHULNA', 'KISHOREGONJ', 'KURIGRAM', 'KUSHTIA', 'LAKSHMIPUR',
       'LALMONIRHAT', 'MADARIPUR', 'MAGURA', 'MANIKGANJ', 'MAULVIBAZAR',
       'MEHERPUR', 'MUNSHIGANJ', 'MYMENSINGH', 'NAOGAON', 'NARAIL',
       'NARAYANGANJ', 'NARSINGDI', 'NATORE', 'NETRAKONA', 'NILPHAMARI',
       'NOAKHALI', 'PABNA', 'PANCHAGARH', 'PATUAKHALI', 'PIROJPUR', 'RAJBARI',
       'RAJSHAHI', 'RANGAMATI', 'RANGPUR', 'SATKHIRA', 'SHARIATPUR', 'SHERPUR',
       'SIRAJGANJ', 'SUNAMGANJ', 'SYLHET', 'TANGAIL', 'THAKURGAON'],
      dtype='object')

In [12]:
df_tmp.district.cat.codes

0         55
1         59
2         13
3         61
4         11
          ..
999995    13
999996    13
999997    15
999998    20
999999    51
Length: 1000000, dtype: int8

In [13]:
# Turn categorical variables into numbers
for label, content in df_tmp.items():
    # Check columns which *aren't* numeric
    if not pd.api.types.is_numeric_dtype(content):
        # Add binary column to inidicate whether sample had missing value
        df_tmp[label+"_is_missing"] = pd.isnull(content)
        # We add the +1 because pandas encodes missing categories as -1
        df_tmp[label] = pd.Categorical(content).codes+1    

In [14]:
df_tmp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 22 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   item_key                1000000 non-null  int64  
 1   store_key               1000000 non-null  int64  
 2   payment_key             1000000 non-null  int64  
 3   trans_type              1000000 non-null  int8   
 4   man_country             1000000 non-null  int8   
 5   descript                1000000 non-null  int8   
 6   supplier                1000000 non-null  int8   
 7   unit                    1000000 non-null  int8   
 8   year                    1000000 non-null  int64  
 9   month                   1000000 non-null  int64  
 10  day                     1000000 non-null  int64  
 11  hour                    1000000 non-null  int64  
 12  week                    1000000 non-null  int8   
 13  district                1000000 non-null  int8   
 14  tot

In [15]:
df_tmp.head().T

,0,1,2,3,4
item_key,131,17,133,23,91
store_key,86,164,640,32,455
payment_key,32,25,19,30,15
trans_type,1,1,1,1,1
man_country,1,2,9,2,2
descript,17,1,18,1,8
supplier,5,10,9,10,10
unit,9,7,9,7,6
year,2020,2016,2018,2017,2018
month,2,10,2,11,4


# Modeling with RandomForest Regressor

In [16]:
# This won't work since we've got missing numbers and categories
from sklearn.ensemble import RandomForestRegressor

In [17]:
%%time
# Instantiate model

model = RandomForestRegressor(n_jobs=-1)

# Fit the model
model.fit(df_tmp.drop("total_price", axis=1), df_tmp.total_price)

CPU times: total: 39min 42s
Wall time: 5min 23s


RandomForestRegressor(n_jobs=-1)

In [44]:
# Score the model
model.score(df_tmp.drop("total_price", axis=1), df_tmp.total_price)

0.9135102359675323

# Splitting data into train/valid sets

In [18]:
# Split data into training and validation
df_val = df_tmp[df_tmp.year == 2021]
df_train = df_tmp[df_tmp.year != 2021]

len(df_val), len(df_train)

(8333, 991667)

In [19]:
# Split data into X & y
X_train, y_train = df_train.drop("total_price", axis=1), df_train.total_price
X_valid, y_valid = df_val.drop("total_price", axis=1), df_val.total_price

X_train.shape, y_train.shape, X_valid.shape, y_valid.shape

((991667, 21), (991667,), (8333, 21), (8333,))

In [ ]:
#Splitting our dataset to Training and Testing dataset
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [20]:
# Create evaluation function (the competition uses Root Mean Square Log Error)
from sklearn.metrics import mean_squared_log_error, mean_absolute_error

def rmsle(y_test, y_preds):
    return np.sqrt(mean_squared_log_error(y_test, y_preds))

# Create function to evaluate our model
def show_scores(model):
    train_preds = model.predict(X_train)
    val_preds = model.predict(X_valid)
    scores = {"Training MAE": mean_absolute_error(y_train, train_preds),
              "Valid MAE": mean_absolute_error(y_valid, val_preds),
              "Training RMSLE": rmsle(y_train, train_preds),
              "Valid RMSLE": rmsle(y_valid, val_preds),
              "Training R^2": model.score(X_train, y_train),
              "Valid R^2": model.score(X_valid, y_valid)}
    return scores

In [27]:
# Change max samples in RandomForestRegressor
model = RandomForestRegressor(n_jobs=-1,
                              max_samples=500000)

In [28]:
%%time
# Cutting down the max number of samples each tree can see improves training time
model.fit(X_train, y_train)

CPU times: total: 19min 17s
Wall time: 2min 41s


RandomForestRegressor(max_samples=500000, n_jobs=-1)

In [29]:
model.score(X_train, y_train)

0.7748125936747305

In [30]:
show_scores(model)

{'Training MAE': 29.53076288797886,
 'Valid MAE': 48.863299531981276,
 'Training RMSLE': 0.5215778046188156,
 'Valid RMSLE': 0.7233693651149063,
 'Training R^2': 0.7748125936747305,
 'Valid R^2': 0.40833190687564735}

## Hyperperameters Tunning

In [31]:
%%time
from sklearn.model_selection import RandomizedSearchCV

# Different RandomForestClassifier hyperparameters
rf_grid = {"n_estimators": np.arange(10, 100, 10),
           "max_depth": [None, 3, 5, 10],
           "min_samples_split": np.arange(2, 20, 2),
           "min_samples_leaf": np.arange(1, 20, 2),
           "max_features": [0.5, 1, "sqrt", "auto"],
           "max_samples": [10000]}

rs_model = RandomizedSearchCV(RandomForestRegressor(),
                              param_distributions=rf_grid,
                              n_iter=20,
                              cv=5,
                              verbose=True)

rs_model.fit(X_train, y_train)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
CPU times: total: 6min
Wall time: 6min 6s


RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(), n_iter=20,
                   param_distributions={'max_depth': [None, 3, 5, 10],
                                        'max_features': [0.5, 1, 'sqrt',
                                                         'auto'],
                                        'max_samples': [10000],
                                        'min_samples_leaf': array([ 1,  3,  5,  7,  9, 11, 13, 15, 17, 19]),
                                        'min_samples_split': array([ 2,  4,  6,  8, 10, 12, 14, 16, 18]),
                                        'n_estimators': array([10, 20, 30, 40, 50, 60, 70, 80, 90])},
                   verbose=True)

In [33]:
# Find the best parameters from the RandomizedSearch 
rs_model.best_params_

{'n_estimators': 80,
 'min_samples_split': 18,
 'min_samples_leaf': 9,
 'max_samples': 10000,
 'max_features': 'auto',
 'max_depth': None}

In [34]:
# Evaluate the RandomizedSearch model
show_scores(rs_model)

{'Training MAE': 48.146628279220224,
 'Valid MAE': 48.289349106168665,
 'Training RMSLE': 0.721491115378132,
 'Valid RMSLE': 0.7202715411412177,
 'Training R^2': 0.41137020673489555,
 'Valid R^2': 0.4283049393753745}

## Train a model with the best parameters

In [37]:
%%time
ideal_model = RandomForestRegressor(n_estimators=80,
                                    min_samples_leaf=9,
                                    min_samples_split=18,
                                    max_features='auto',
                                    max_samples=10000,
                                    max_depth=None)
ideal_model.fit(X_train, y_train)

CPU times: total: 6.8 s
Wall time: 6.79 s


RandomForestRegressor(max_samples=10000, min_samples_leaf=9,
                      min_samples_split=18, n_estimators=80)

## Save model to disk

In [41]:
import pickle

In [42]:
filename = "ideal_model.sav"
pickle.dump(ideal_model, open(filename, 'wb'))

In [38]:
show_scores(ideal_model)

{'Training MAE': 48.154483098552845,
 'Valid MAE': 48.37970721119038,
 'Training RMSLE': 0.7218697002445341,
 'Valid RMSLE': 0.7211529887130963,
 'Training R^2': 0.4110508692030509,
 'Valid R^2': 0.4277854446939938}

In [43]:
%%time
# Faster model
fast_model = RandomForestRegressor(n_estimators=40,
                                   min_samples_leaf=3,
                                   max_features=0.5,
                                   n_jobs=-1)
fast_model.fit(X_train, y_train)

CPU times: total: 6min 4s
Wall time: 48.4 s


RandomForestRegressor(max_features=0.5, min_samples_leaf=3, n_estimators=40,
                      n_jobs=-1)

In [44]:
show_scores(fast_model)

{'Training MAE': 29.913226308845285,
 'Valid MAE': 48.928652460028374,
 'Training RMSLE': 0.5285396518559204,
 'Valid RMSLE': 0.7240002252492043,
 'Training R^2': 0.7649770210023219,
 'Valid R^2': 0.4042742938014814}

In [45]:
fast_model.score(X_train, y_train)

0.7649770210023219

In [53]:
# Let's see how the model goes predicting on the test data
y_preds =fast_model.predict(X_valid)
y_preds

array([ 86.59345058,  47.86396825,  79.60379509, ...,  91.01800595,
       119.13322421,  92.03171131])

### Predict One Year(2021) Sales For each of the items

In [55]:
df_preds = pd.DataFrame()
df_preds["item_key"] = X_valid["item_key"]
df_preds["total_sales"]= y_preds
df_preds

,item_key,total_sales
20,229,86.593451
56,45,47.863968
100,75,79.603795
154,210,88.249924
409,13,47.336853
...,...,...
999726,112,67.531365
999738,215,92.560955
999758,163,91.018006
999828,169,119.133224


***Another model***

In [83]:
%%time
model2 = RandomForestRegressor(n_estimators=40,
                                   min_samples_leaf=3,
                                   max_features=0.24,
                                   n_jobs=-1)
model2.fit(X_train, y_train)

CPU times: total: 4min
Wall time: 32.4 s


RandomForestRegressor(max_features=0.24, min_samples_leaf=3, n_estimators=40,
                      n_jobs=-1)

In [84]:
model2.score(X_train, y_train)

0.7133514536475094

In [86]:
%%time
model3 = RandomForestRegressor(n_estimators=40,
                                   min_samples_leaf=3,
                                   max_features=0.1,
                                   n_jobs=-1)
model3.fit(X_train, y_train)

CPU times: total: 2min 39s
Wall time: 22.1 s


RandomForestRegressor(max_features=0.1, min_samples_leaf=3, n_estimators=40,
                      n_jobs=-1)

In [87]:
model3.score(X_train, y_train)

0.6512691683754341

# Modeling with Linear regression 

In [58]:
from sklearn.linear_model import LinearRegression

In [59]:
%%time
# Instantiate model

model_Lr = LinearRegression()

# Fit the model
model_Lr.fit(df_tmp.drop("total_price", axis=1), df_tmp.total_price)

CPU times: total: 1.98 s
Wall time: 4.88 s


LinearRegression()

In [60]:
# Score the model
model_Lr.score(df_tmp.drop("total_price", axis=1), df_tmp.total_price)

0.008379412526208219

In [61]:
# Split data into training and validation
df_val = df_tmp[df_tmp.year == 2021]
df_train = df_tmp[df_tmp.year != 2021]

len(df_val), len(df_train)

(8333, 991667)

In [62]:
# Split data into X & y
X_train, y_train = df_train.drop("total_price", axis=1), df_train.total_price
X_valid, y_valid = df_val.drop("total_price", axis=1), df_val.total_price

X_train.shape, y_train.shape, X_valid.shape, y_valid.shape

((991667, 21), (991667,), (8333, 21), (8333,))

In [63]:
# Fit the model
model_Lr.fit(X_train,y_train)

LinearRegression()

In [65]:
model_Lr.score(X_train,y_train)

0.008385944021533365

In [66]:
# Let's see how the model goes predicting on the test data
y_preds =model_Lr.predict(X_valid)
y_preds

array([109.07329518, 102.74931505, 117.21039862, ..., 108.76653919,
       102.37869414, 117.21685801])

In [90]:
# Instantiate model

model_Lr2 = LinearRegression(copy_X=True, n_jobs=-1,fit_intercept=True)

# Fit the model
model_Lr2.fit(X_train,y_train)

LinearRegression(n_jobs=-1)

In [91]:
y_preds =model_Lr2.predict(X_valid)
y_preds

array([109.07329518, 102.74931505, 117.21039862, ..., 108.76653919,
       102.37869414, 117.21685801])

In [92]:
#Calculating the Coefficients
model_Lr2.coef_

array([ 3.91530220e-01,  1.39813832e-04, -5.03143342e-03, -2.37327588e-02,
        1.80404537e+00, -2.79832306e+00,  4.74165963e-01, -5.32671418e-01,
        3.15244810e-02,  4.59164844e-02, -4.37367435e-03,  5.05371769e-03,
       -2.31207591e-02,  5.81480322e-03,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00])

In [72]:
#Calculating the Intercept
model_Lr2.intercept_

24.093698641373905

In [ ]:
#Calculating the R squared value
from sklearn.metrics import r2_score
r2_score(y_valid, y_preds)
